In [1]:
# ==============================================================================
# 1. IMPORTS Y CONFIGURACIÓN
# ==============================================================================
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from google.cloud import storage

# Nota: win32com.client es una dependencia específica de Windows para manejar Excel
try:
    import win32com.client as win32
except ImportError:
    print("Advertencia: win32com.client no encontrado. La conversión de .xls a .xlsx no funcionará.")
    win32 = None

# Configuración Centralizada
CONFIG = {
    # Web Automation (SICOE)
    "SICOE_URL": "https://sicoe.com.co/sicoe/dist/#/",
    "SICOE_NIT": "8301256101",
    "SICOE_USER": "analistadatos",
    "SICOE_PASS": "SEna12345*",
    "DOWNLOAD_DIR": r"C:\Users\Lenovo\Downloads",
    "ADVERTENCIA_BUTTON_SELECTOR": ".bg-blue-950.text-slate-50",
    "REPORT_FILE_PREFIX": "rutas_x_asesor",
    "WAIT_TIMEOUT": 15,
    # Google Cloud Storage
    "GCS_CREDENTIALS_PATH": 'credentials/croc-454221-e1a3c2e02181.json',
    "GCS_BUCKET_NAME": 'bucket-quickstart_croc_830',
    "GCS_DEST_FOLDER": 'raw/Ventas/sicoe_rutas_vendedor',
}
print("Se han cargado las configuraciones.")

# ==============================================================================
# 2. CLASE SicoeAutomator
# ==============================================================================
class SicoeAutomator:
    """Clase para automatizar las interacciones con la plataforma SICOE."""
    
    def __init__(self, config):
        self.config = config
        self.driver = None
        self._setup_driver()

    def _setup_driver(self):
        """Inicializa el WebDriver de Chrome."""
        print("🔧 Configurando e iniciando navegador...")
        options = webdriver.ChromeOptions()
        options.add_argument("--start-maximized")
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_experimental_option('prefs', {
            "download.default_directory": self.config["DOWNLOAD_DIR"],
            "download.prompt_for_download": False,
            "download.directory_upgrade": True,
            "safebrowsing.enabled": True
        })

        self.driver = webdriver.Chrome(
            service=Service(ChromeDriverManager().install()), 
            options=options
        )
        print("✅ Navegador iniciado.")

    def wait_for_element(self, by: By, value: str, timeout: int = None):
        """Espera hasta que un elemento esté presente y lo devuelve."""
        timeout = timeout or self.config["WAIT_TIMEOUT"]
        try:
            return WebDriverWait(self.driver, timeout).until(
                EC.presence_of_element_located((by, value))
            )
        except TimeoutException:
            print(f"❌ Tiempo de espera agotado al buscar elemento {by}={value}")
            return None
        except Exception as e:
            print(f"❌ Error al esperar elemento {by}={value}: {e}")
            return None

    def login_and_navigate(self):
        """Realiza el proceso de login y navegación hasta el menú de informes."""
        print(f"🌐 Cargando URL: {self.config['SICOE_URL']}")
        self.driver.get(self.config["SICOE_URL"])

        # 1. Aceptar Modal
        print("--> Esperando el selector de Aceptar el modal...")
        advertencia_button = self.wait_for_element(By.CSS_SELECTOR, self.config["ADVERTENCIA_BUTTON_SELECTOR"])
        if advertencia_button:
            advertencia_button.click()
            time.sleep(1) # Espera pequeña para que el modal se cierre

        # 2. Click en 'Iniciar sesión'
        print("--> Click en 'Iniciar sesión'")
        boton_login = WebDriverWait(self.driver, self.config["WAIT_TIMEOUT"]).until(
            EC.element_to_be_clickable((By.XPATH, '//a[contains(text(), "Iniciar sesión")]'))
        )
        boton_login.click()

        # 3. Ingreso de credenciales
        print("--> Ingresando credenciales...")
        self.wait_for_element(By.ID, "nit").send_keys(self.config["SICOE_NIT"])
        self.wait_for_element(By.ID, "login").send_keys(self.config["SICOE_USER"])
        self.wait_for_element(By.ID, "passwd").send_keys(self.config["SICOE_PASS"])

        # 4. Manejo de Checkbox y Login
        checkbox = self.wait_for_element(By.CSS_SELECTOR, 'input[type="checkbox"]')
        if checkbox:
            checkbox.click()
            
        boton_iniciar = WebDriverWait(self.driver, self.config["WAIT_TIMEOUT"]).until(
            EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Iniciar")]'))
        )
        boton_iniciar.click()
        time.sleep(3) # Espera después del login

        # 5. Navegación al Informe
        print("--> Navegando a Informes y luego a Rutas...")
        boton_informes = self.wait_for_element(By.CSS_SELECTOR, 'a[href="/sam/menu1/menu/index.php"]')
        if boton_informes:
            boton_informes.click()

        boton_next = self.wait_for_element(By.CSS_SELECTOR, 'a.buttonNext')
        if boton_next:
            boton_next.click()

    def download_rutas_report(self):
        """Navega y dispara la descarga del informe 'Rutas por Vendedor'."""
        try:
            rutas_por_vendedor = self.wait_for_element(
                By.XPATH, 
                f"//a[@class='tip' and contains(@href, '{self.config['REPORT_FILE_PREFIX']}')]"
            )
            if rutas_por_vendedor:
                rutas_por_vendedor.click()
                print("✅ Click en 'Rutas por Vendedor'.")

                # 6. Esperar y cambiar al iframe
                WebDriverWait(self.driver, self.config["WAIT_TIMEOUT"]).until(
                    EC.frame_to_be_available_and_switch_to_it((By.ID, "sb-player"))
                )
                print("✅ Cambiado al iframe con el informe.")
                
                # 7. Click en Exportar a Excel
                boton_excel = WebDriverWait(self.driver, self.config["WAIT_TIMEOUT"]).until(
                    EC.element_to_be_clickable((By.ID, "excel"))
                )
                boton_excel.click()
                print("✅ Click en Exportar a Excel. Esperando descarga...")

                time.sleep(25)  # Espera para asegurar que la descarga .xls se complete
                self.driver.quit()
                print("✅ Navegador cerrado.")
                return True
            return False
        except Exception as e:
            print(f"❌ Error durante la descarga del informe: {e}")
            if self.driver:
                self.driver.quit()
            return False

# ==============================================================================
# 3. FUNCIONES DE UTILIDAD PARA ARCHIVOS
# ==============================================================================

def find_latest_file(directory: str, prefix: str, extension: str) -> str or None:
    """Busca y retorna la ruta completa del archivo más reciente con el prefijo y extensión dados."""
    archivos = [
        f for f in os.listdir(directory)
        if prefix in f and f.endswith(extension)
    ]
    if not archivos:
        return None

    # Ordenar por tiempo de modificación (el último es el más reciente)
    archivos.sort(key=lambda f: os.path.getmtime(os.path.join(directory, f)))
    archivo = archivos[-1]
    ruta_completa = os.path.join(directory, archivo)
    return ruta_completa

def convert_xls_to_xlsx(ruta_xls: str) -> str or None:
    """Convierte un archivo .xls a .xlsx usando la librería win32com."""
    if win32 is None:
        print("❌ Conversión fallida. win32com.client no está instalado.")
        return None
        
    try:
        excel = win32.gencache.EnsureDispatch('Excel.Application')
        wb = excel.Workbooks.Open(ruta_xls)
        nueva_ruta = ruta_xls + "x"  # Agrega una "x" para .xlsx
        wb.SaveAs(nueva_ruta, FileFormat=51)  # 51 = formato .xlsx
        wb.Close()
        excel.Quit()
        print("✅ Archivo convertido a:", nueva_ruta)
        return nueva_ruta
    except Exception as e:
        print(f"❌ Error al convertir el archivo {ruta_xls}: {e}")
        return None

# ==============================================================================
# 4. CLASE GCSUploader
# ==============================================================================

class GCSUploader:
    """Clase para manejar la subida de archivos a Google Cloud Storage (GCS)."""
    
    def __init__(self, config):
        self.config = config
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = self.config["GCS_CREDENTIALS_PATH"]
        self.client = storage.Client()
        self.bucket = self.client.bucket(self.config["GCS_BUCKET_NAME"])
        self.dest_folder = self.config["GCS_DEST_FOLDER"]

    def _clean_destination_folder(self):
        """Elimina todos los archivos en la carpeta de destino del bucket."""
        print(f"🧹 Limpiando carpeta '{self.dest_folder}' en el bucket '{self.bucket.name}'...")
        blobs = self.bucket.list_blobs(prefix=self.dest_folder)
        count = 0
        for blob in blobs:
            # Añadir una condición para evitar borrar la carpeta raíz si no hay archivos
            if blob.name != self.dest_folder and not blob.name.endswith('/'):
                print(f"🗑 Eliminando: {blob.name}")
                blob.delete()
                count += 1
                time.sleep(1) # Pequeña espera para evitar errores de tasa
        print(f"✅ Eliminados {count} archivos en GCS.")

    def upload_file(self, file_path: str):
        """Sube un archivo, limpiando previamente la carpeta de destino."""
        
        # 1. Limpiar carpeta de destino
        self._clean_destination_folder()
        
        # 2. Subir el nuevo archivo
        file_name = os.path.basename(file_path)
        nombre_archivo_bucket = os.path.join(self.dest_folder, file_name).replace("\\", "/")
        
        print(f"📤 Subiendo archivo local: {file_name} a GCS: {nombre_archivo_bucket}...")
        
        blob = self.bucket.blob(nombre_archivo_bucket)
        blob.upload_from_filename(file_path)
        
        print(f"✅ Archivo subido exitosamente: {nombre_archivo_bucket}")
        time.sleep(6) # Espera para asegurar que la subida se complete

# ==============================================================================
# 5. FUNCIÓN DE LIMPIEZA LOCAL
# ==============================================================================

def clean_local_download_folder(directory: str, prefix: str):
    """Elimina archivos con el prefijo dado en la carpeta de descargas."""
    print(f"\n🧹 Iniciando limpieza local en: {directory}")
    archivos_eliminados = 0
    for archivo in os.listdir(directory):
        if prefix in archivo:
            ruta_completa = os.path.join(directory, archivo)
            try:
                os.remove(ruta_completa)
                print(f"🗑 Eliminado: {ruta_completa}")
                archivos_eliminados += 1
            except Exception as e:
                print(f"⚠️ Error al eliminar {archivo}: {e}")

    if archivos_eliminados == 0:
        print("📁 No se encontraron archivos para eliminar.")
    else:
        print(f"✅ Eliminados {archivos_eliminados} archivos locales.")

# ==============================================================================
# 6. FUNCIÓN PRINCIPAL (ORQUESTACIÓN)
# ==============================================================================

def main():
    """Flujo principal de la descarga, conversión y subida del informe de rutas."""
    
    # --- A. DESCARGA DEL INFORME ---
    automator = SicoeAutomator(CONFIG)
    
    try:
        automator.login_and_navigate()
        if not automator.download_rutas_report():
            print("❌ El proceso de descarga no se completó con éxito. Terminando.")
            return

        # --- B. PROCESAMIENTO LOCAL ---
        
        # 1. Buscar el archivo .xls recién descargado
        ruta_xls = find_latest_file(CONFIG["DOWNLOAD_DIR"], CONFIG["REPORT_FILE_PREFIX"], ".xls")
        if not ruta_xls:
            print("❌ No se encontró el archivo .xls descargado. Terminando.")
            return

        print("✅ Archivo .xls encontrado:", ruta_xls)
        
        # 2. Convertir a .xlsx
        ruta_xlsx = convert_xls_to_xlsx(ruta_xls)
        if not ruta_xlsx:
            print("❌ La conversión a .xlsx falló. Terminando.")
            return

        # Opcional: Cargar y procesar el DataFrame (solo se lee para verificar)
        df = pd.read_excel(ruta_xlsx)
        print(f"✅ DataFrame leído exitosamente. Filas: {len(df)}")


        # --- C. SUBIDA A GCS ---
        uploader = GCSUploader(CONFIG)
        uploader.upload_file(ruta_xlsx)

    except Exception as e:
        print(f"\nFATAL: Ocurrió un error en el flujo principal: {e}")
    finally:
        # --- D. LIMPIEZA FINAL ---
        clean_local_download_folder(CONFIG["DOWNLOAD_DIR"], CONFIG["REPORT_FILE_PREFIX"])


if __name__ == "__main__":
    main()

Se han cargado las configuraciones.
🔧 Configurando e iniciando navegador...
✅ Navegador iniciado.
🌐 Cargando URL: https://sicoe.com.co/sicoe/dist/#/
--> Esperando el selector de Aceptar el modal...
--> Click en 'Iniciar sesión'

FATAL: Ocurrió un error en el flujo principal: Message: 
Stacktrace:
	GetHandleVerifier [0x0xbdc333+65459]
	GetHandleVerifier [0x0xbdc374+65524]
	(No symbol) [0x0x9fd973]
	(No symbol) [0x0xa476e7]
	(No symbol) [0x0xa47a8b]
	(No symbol) [0x0xa8dea2]
	(No symbol) [0x0xa69e44]
	(No symbol) [0x0xa8b606]
	(No symbol) [0x0xa69bf6]
	(No symbol) [0x0xa3b38e]
	(No symbol) [0x0xa3c274]
	GetHandleVerifier [0x0xe5eda3+2697763]
	GetHandleVerifier [0x0xe59ec7+2677575]
	GetHandleVerifier [0x0xc04194+228884]
	GetHandleVerifier [0x0xbf49f8+165496]
	GetHandleVerifier [0x0xbfb18d+192013]
	GetHandleVerifier [0x0xbe47d8+99416]
	GetHandleVerifier [0x0xbe4972+99826]
	GetHandleVerifier [0x0xbcebea+10346]
	BaseThreadInitThunk [0x0x766afcc9+25]
	RtlGetAppContainerNamedObjectPath [0x0x77